# 유방암 예측
_**유방 질량 이미지에서 파생된 feature 로 SageMaker의 linear-learner 를 사용하여 유방암 예측**_

---

## 차례

1. [배경](#배경)
1. [설정](#설정)
1. [데이터](#데이터)
1. [훈련](#훈련)
1. [호스팅](#호스팅)
1. [예측](#예측)
1. [확장](#확장)

---

## 배경
이 노트북은 예측을 위해 `linear models` 을 요구하는 문제해결 애플리케이션을 위해 Sagemaker 의 알고리즘을 사용하는 법을 보여줍니다. <br />
이 예에서 우리는 유방암 예측을 위해 UCI 의 유방암 진단 데이터 셋을 가져옵니다. <br />
목표는 이 데이터 세트를 사용하여 유방종 이미지가 양성 또는 악성 종양을 나타내는지 여부에 대한 예측 모델을 구축하는 것입니다. <br />

* Sagemaker 사용을 위한 기본적인 환경 구축
* Sagemaker 알고리즘이 사용할 수 있도록 데이터셋을 protobuf 포맷으로 변환하고 S3 에 업로드
* SageMaker linear learner 알고리즘으로 훈련
* 훈련된 모델을 호스팅
* 훈련된 모델을 사용하여 스코어링

필요한 Python 라이브러리를 import 합니다. 

In [1]:
import os                                         # For manipulating filepath names  
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import io
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.

import sagemaker                                  # Amazon SageMaker's Python SDK provides many helper functions
from sagemaker import get_execution_role          # Define IAM role
import sagemaker.amazon.common as smac
import boto3

## 설정

다음을 설정하는 것으로 시작 해보겠습니다.

* SageMaker Role은 데이터에 대한 학습 및 호스팅 액세스 권한을 부여하는 데 사용됩니다. <br />
  아래 코드는 SageMaker 노트북 인스턴스에서 사용하는 것과 동일한 역할을 사용합니다. 그렇지 않은 경우 SageMakerFullAccess 정책이 연결된 역할의 전체 ARN을 지정합니다.
* 모델 객체를 훈련하고 저장하는 데 사용하려는 S3 버킷 지정

In [2]:


role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket and prefix
#bucket = sagemaker.Session().default_bucket()
#bucket = '{ENTER_BUCKET_NAME}'            # 20220422-sagemaker-workshop-jbuh-kwpark
bucket = '20220422-sagemaker-workshop-jbuh-kwpark'            # 20220422-sagemaker-workshop-jbuh-kwpark
prefix = 'sagemaker/breast-cancer'  # place to upload training files within the bucket


---
## 데이터


> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository ( http://archive.ics.uci.edu/ml ). Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set ( https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic) ).

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set ( https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data )
다음 코드는 위 데이터 소스 위치로부터 데이터를 로컬 폴더에 다운로드하여 저장한 후에 파일의 이름을 data.csv 로 변경합니다. 이 작업은 조금 시간이 소요될 수 있습니다. 

In [3]:
s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file("sagemaker-sample-files", "datasets/tabular/breast_cancer/wdbc.csv", filename)
data = pd.read_csv(filename, header=None)

# 컬럼명 지정
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# 데이터 저장
data.to_csv("data.csv", sep=",", index=False)

# 데이터 파일의 모양(shape) 프린트
print(data.shape)

# 최상위 몇개의 행 표시 
display(data.head())

# 데이터 객체를 표시
display(data.describe())

# we will also summarize the categorical field diganosis
# 필드 분류를 요약 
display(data.diagnosis.value_counts())

(569, 32)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


B    357
M    212
Name: diagnosis, dtype: int64

#### 주요 관찰 결과:
* 데이터는 569개의 관측값과 32개의 컬럼을 가지고 있습니다. 
* 첫번째 필드는 'id' 입니다.
* 두번째 필드는 'diagnosis' 이고 악성('M' = Malignant)과 양성('B' = Benign)에 대한 진단 지표입니다.
* 예측에 활용할 수 있는 30개의 다른 숫자형 feature (특성값) 들이 있습니다. 

## Features 와 Labels 생성
#### 아래 코드는 데이터를 80% 의 training 데이터, 10% 의 validation 용 데이터, 10% test 용 데이터로 나눕니다. 

In [4]:
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
test_list = rand_split >= 0.9

data_train = data[train_list]
data_val = data[val_list]
data_test = data[test_list]

train_y = ((data_train.iloc[:, 1] == "M") + 0).to_numpy()
train_X = data_train.iloc[:, 2:].to_numpy()

val_y = ((data_val.iloc[:, 1] == "M") + 0).to_numpy()
val_X = data_val.iloc[:, 2:].to_numpy()

test_y = ((data_test.iloc[:, 1] == "M") + 0).to_numpy()
test_X = data_test.iloc[:, 2:].to_numpy();


이제 데이터 세트를 Amazon SageMaker 알고리즘에서 사용하는 recordIO-wrapped protobuf 형식으로 변환한 다음 이 데이터를 S3에 업로드합니다. 

In [5]:
train_file = "linear_train.data"

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, train_X.astype("float32"), train_y.astype("float32"))
f.seek(0)

boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train", train_file)
).upload_fileobj(f)


다음으로 유효성 검사 데이터 세트를 변환하고 업로드합니다.

In [6]:
validation_file = "linear_validation.data"

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, val_X.astype("float32"), val_y.astype("float32"))
f.seek(0)

boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "validation", validation_file)
).upload_fileobj(f)

---
## 훈련

Amazon SageMaker의 Linear Learner는 실제로 각각 약간 다른 하이퍼파라미터를 사용하여 여러 모델을 병렬로 맞춘 다음 가장 잘 맞는 모델을 반환합니다. <br />
이 기능은 자동으로 활성화됩니다. 다음과 같은 매개변수를 사용하여 이에 영향을 줄 수 있습니다. <br />

- 'num_models' : 모델 실행의 전체 수. 최적에 가까운 솔루션을 찾기 위해 가까운 매개변수 값을 가진 모델을 선택합니다. 이번 실습에서는 최대값인 32를 사용합니다.
- 'loss' : 모델 추정치의 실수에 페널티를 부여하는 방법을 제어. 이번 실습에서는 데이터를 정리하는 데 많은 시간을 소비하지 않았으므로 absolute loss 을 사용하고 absolute loss 은 이상값 (outlier) 에 덜 민감합니다.
- 'wd' 또는 'l1' : 정규화를 제어. 정규화는 우리의 추정치가 훈련 데이터에 너무 미세하게 조정되는 것을 방지함으로써 모델 과적합을 방지할 수 있습니다. 이번 실습에서는 이 매개변수를 기본 "자동"으로 남겨둘 것입니다.


### SageMaker 의 linear-learner 를 훈련하고 호스팅하기 위해 사용되는 컨테이너 이미지 지정

SageMaker 의 ECR 컨테이너를 통해 해당되는 built-in 알고리즘을 사용할 수 있습니다.

In [7]:
from sagemaker import image_uris

container = image_uris.retrieve(framework="linear-learner", 
                                region=boto3.Session().region_name)

### SageMaker의 Estimator 생성

다음으로 파라미터를 지정하여 esitmator를 생성합니다.

- built-in 알고리즘 컨테이너 사용
- 사용할 IAM 역할(role)
- 학습용 인스턴스 타입과 수량 ('local_cpu'를 사용해 해당 노트북의 인스턴스 내에서 학습을 진행할 수도 있습니다.)
- 출력 데이터를 위한 S3위치
- 알고리즘 하이퍼파라미터

이제 다음 파라미터를 이용하여 .fit() 명령을 실행합니다.

- 학습용(train)/검증용(validation) 데이터가 있는 S3 위치

본 예제는 학습과 검증 데이터셋을 모두 사용하므로 두 채널을 모두 지정합니다. Trainin job을 수행하기 위해서 학습용 서버가 생성되는데에 5분정도 소요됩니다.

데이터 학습을 수행할 시에 발생하는 과금은, EC2 인스턴스의 생성 시간이 제외된 데이터를 학습하는 시간만 요금이 부과됩니다. Log의 마지막 부분에 표시되는 Training seconds와 Billable seconds를 참고하셔서 과금이 발생한 시간을 참고하실 수 있습니다.

- Training seconds: Training job을 실행한 실제 컴퓨팅 학습 시간
- Billable seconds: Spot 할인이 적용된 후 청구되는 시간

In [8]:
sess = sagemaker.Session()

linear_job = "breast-cancer-linear-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print("Job name is:", linear_job)

train_file = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix))
validation_file = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix))

linear = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m5.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess,
                                    base_job_name=linear_job)

linear.set_hyperparameters(feature_dim=30,
                        mini_batch_size=100,
                        predictor_type='regressor',
                        epochs=10,
                        num_models=32,
                        loss='absolute_loss')

linear.fit({'train': train_file, 'validation': validation_file})

Job name is: breast-cancer-linear-2022-04-21-18-40-29
2022-04-21 18:40:30 Starting - Starting the training job...
2022-04-21 18:40:53 Starting - Preparing the instances for trainingProfilerReport-1650566429: InProgress
......
2022-04-21 18:41:54 Downloading - Downloading input data...
2022-04-21 18:42:14 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/21/2022 18:42:44 INFO 139912272332608 integration.py:636] worker started
[04/21/2022 18:42:44 INFO 139912272332608] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.0

## 모델 호스팅

이제 모델을 학습시켰으므로 Amazon SageMaker의 실시간 호스팅 엔드포인트에 모델을 배포할 수 있습니다. 이렇게 하면 모델에서 동적으로 예측(또는 추론)할 수 있습니다. Endpoint 생성은 5-10분 정도 소요됩니다.

- 호스팅에 사용할 EC2 인스턴스 유형
- 인스턴스의 초기 갯수
- 호스팅 모델명

In [9]:
%%time

endpoint_name='breast-cancer-'+strftime("%Y-%m-%d-%H-%M-%S", gmtime())

linear_endpoint = linear.deploy(initial_instance_count=1,
                           instance_type='ml.m5.xlarge',
                           endpoint_name=endpoint_name)

-----!CPU times: user 92.5 ms, sys: 9.19 ms, total: 102 ms
Wall time: 2min 31s


## 모델 검증(예측)
### 테스트 데이터를 이용한 예측

이제 호스팅된 엔드포인트가 있으므로 여기에서 통계적 예측을 생성할 수 있습니다. <br />
모델이 얼마나 정확한지 이해하기 위해 테스트 데이터 세트에서 예측해 보겠습니다.
분류 정확도를 측정하기 위한 많은 메트릭이 있습니다. 일반적인 예에는 다음이 포함됩니다. 

- Precision (정밀도)
- Recall
- F1 measure (F1 측정값)
- Area under the ROC curve - AUC
- Total Classification Accuracy (총 분류정확도)
- Mean Absolute Error (평균절대오차)

이 예에서 우리는 작업을 단순하게 유지하기 위해 Total Classification Accuracy 를 선택 지표로 사용할 것입니다. <br/>
또한 linear learner 가 이 메트릭을 사용하여 최적화되었기 때문에 MAE(Mean Absolute Error)를 평가할 것입니다. 

### array 를 csv 로 변환하기 위한 함수

In [10]:
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=",", fmt="%g")
    return csv.getvalue().decode().rstrip()


예측값을 얻기 위해 endpoint 호출

In [11]:
runtime = boto3.client("runtime.sagemaker")

payload = np2csv(test_X)
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)
result = json.loads(response["Body"].read().decode())
test_pred = np.array([r["score"] for r in result["predictions"]])


모든 인스턴스를 예측하기 위해 대다수 클래스를 사용하는 기준선 평균절대예측오류와 선형학습자 기반 평균절대예측오류를 비교합니다. <br/>
- 기준선 기반 평균절대예측오류(Baseline Mean Abolute Error) - Baseline MAE <br/>
- 선형학습자 기반 평균절대예측오류(Linear Mean Absolute Error) - Linear MAE

In [12]:
test_mae_linear = np.mean(np.abs(test_y - test_pred))
test_mae_baseline = np.mean(
    np.abs(test_y - np.median(train_y))
)  ## training median as baseline predictor

print("Test MAE Baseline :", round(test_mae_baseline, 3))
print("Test MAE Linear:", round(test_mae_linear, 3))

Test MAE Baseline : 0.377
Test MAE Linear: 0.212



예측에 대해 0.5의 분류 임계값을 사용하여 예측 정확도를 비교하고 훈련 데이터 세트로부터 다수 클래스 예측과 비교해 보겠습니다.

In [13]:
test_pred_class = (test_pred > 0.5) + 0
test_pred_baseline = np.repeat(np.median(train_y), len(test_y))

prediction_accuracy = np.mean((test_y == test_pred_class)) * 100
baseline_accuracy = np.mean((test_y == test_pred_baseline)) * 100

print("Prediction Accuracy:", round(prediction_accuracy, 1), "%")
print("Baseline Accuracy:", round(baseline_accuracy, 1), "%")

Prediction Accuracy: 88.5 %
Baseline Accuracy: 62.3 %


## Endpoint 삭제

In [14]:
linear_endpoint.delete_endpoint()

---
## 확장

- 우리의 linear 모델은 유방암을 잘 예측하고 92%에 가까운 전체 정확도를 가지고 있습니다. 하이퍼 파라미터, 손실 함수 등의 다른 값으로 모델을 다시 실행하고 예측이 개선되는지 확인할 수 있습니다. <br/> 
  이러한 하이퍼 파라미터에 대한 추가 조정으로 모델을 다시 실행하면 더 정확한 샘플 외 예측을 제공할 수 있습니다.
- 우리는 또한 많은 feature engineering 을 하지 않았습니다. 여러 feature 의 외적/상호작용을 고려하여 추가 feature 를 생성할 수 있습니다. 
- 추가 확장으로 XGBoost, MXNet 등과 같은 SageMaker를 통해 사용할 수 있는 많은 비선형 모델을 사용할 수 있습니다.